In [1]:
# =============================================================================
# NOTEBOOK 3: MOTOR DE RECOMENDACIÓN FINAL
# =============================================================================

# --- 1. Importación de Librerías ---
import pandas as pd
import numpy as np
import joblib  # Para cargar el modelo y otros artefactos
import random  # Para seleccionar un cliente de ejemplo

print("--- Iniciando Notebook del Motor de Recomendación ---")



--- Iniciando Notebook del Motor de Recomendación ---


In [2]:
# --- 2. Carga de Artefactos Necesarios ---
# Cargamos todo lo que preparamos en los notebooks anteriores.
print("Cargando artefactos necesarios...")
try:
    # Cargar el modelo final (el ajustado)
    modelo = joblib.load('modelo_recomendacion_final.pkl')
    # Cargar la lista de columnas con la que se entrenó el modelo
    training_columns = joblib.load('training_columns.pkl')
    # Cargar los datos originales para obtener información de productos y clientes
    df_articles = pd.read_csv('Datos/articles.csv')
    df_customers = pd.read_csv('Datos/customers.csv')
    df_trans = pd.read_csv('Datos/young_female_trans.csv')
    print("Artefactos y datos cargados exitosamente.")
except FileNotFoundError as e:
    print(f"Error: No se encontró el archivo {e.filename}.")
    print("Asegúrate de que los archivos de datos y los artefactos guardados (.pkl) están en el directorio.")
    exit()

Cargando artefactos necesarios...


c:\Users\crist\AppData\Local\Programs\Python\Python310\lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Artefactos y datos cargados exitosamente.


In [3]:
# --- 3. Preparación de Datos Auxiliares ---
# Re-creamos los perfiles de usuario y características de artículos.
# Esto es necesario para construir los vectores de predicción.
print("\nPreparando perfiles de usuario y características de artículos...")

# Combinar dataframes (versión simplificada del notebook 1)
merged_df = pd.merge(df_trans, df_customers, on='customer_id', how='left')
full_df = pd.merge(merged_df, df_articles, on='article_id', how='left')
full_df['age'].fillna(full_df['age'].median(), inplace=True)
full_df['customer_id'] = full_df['customer_id'].astype(str)
full_df['article_id'] = full_df['article_id'].astype(str)

# Crear perfiles de usuario
user_profiles = full_df.groupby('customer_id').agg(
    age=('age', 'first'),
    avg_price_paid=('price', 'mean'),
    total_articles_bought=('article_id', 'count'),
    fav_product_type=('product_type_name', lambda x: x.mode()[0] if not x.empty else 'Unknown'),
    fav_color=('colour_group_name', lambda x: x.mode()[0] if not x.empty else 'Unknown'),
    fav_department=('department_name', lambda x: x.mode()[0] if not x.empty else 'Unknown')
).reset_index()

# Crear características de artículos
item_popularity = full_df.groupby('article_id').agg(times_purchased=('customer_id', 'count')).reset_index()
# Seleccionamos las mismas columnas que en el notebook 2 para consistencia
static_features = df_articles[['article_id', 'product_type_name', 'product_group_name', 'graphical_appearance_name', 
                                'colour_group_name', 'department_name', 'index_name', 'section_name', 'garment_group_name']].astype({'article_id': str})
final_item_features = pd.merge(static_features, item_popularity, on='article_id', how='left').fillna(0)

# Conjunto de todos los artículos y diccionario de compras por usuario
all_articles = set(df_articles['article_id'].astype(str))
purchased_items_dict = df_trans.groupby('customer_id')['article_id'].apply(set)

del merged_df, full_df # Liberar memoria
import gc
gc.collect()
print("Perfiles y características listos.")


Preparando perfiles de usuario y características de artículos...


C:\Users\crist\AppData\Local\Temp\ipykernel_996\564352538.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_df['age'].fillna(full_df['age'].median(), inplace=True)


Perfiles y características listos.


In [4]:
# --- 4. Función Principal de Recomendación ---
# Esta función encapsula toda la lógica para generar recomendaciones para un solo cliente.
def generar_recomendaciones(customer_id, modelo, n_recommendations=10):
    """
    Genera una lista de N recomendaciones personalizadas para un cliente específico.
    """
    print(f"\n==========================================================")
    print(f"INICIANDO GENERACIÓN DE RECOMENDACIONES PARA EL CLIENTE: {customer_id}")
    print(f"==========================================================")
    
    # 1. Obtener el perfil del usuario a partir del ID
    user_profile = user_profiles[user_profiles['customer_id'] == customer_id]
    if user_profile.empty:
        return f"Error: Cliente con ID '{customer_id}' no encontrado."
        
    # 2. Obtener la lista de artículos que el cliente NO ha comprado (Candidatos)
    items_comprados_por_usuario = purchased_items_dict.get(customer_id, set())
    articulos_candidatos = list(all_articles - items_comprados_por_usuario)
    
    # Para eficiencia, si hay demasiados candidatos, tomamos una muestra aleatoria.
    # Esto simula recomendar a partir de un subconjunto del catálogo (ej. novedades, populares).
    if len(articulos_candidatos) > 5000:
        articulos_candidatos = random.sample(articulos_candidatos, 5000)
    print(f"Paso 1: Se evaluarán {len(articulos_candidatos)} artículos candidatos.")
        
    # 3. Crear el DataFrame para la predicción
    df_predict = pd.DataFrame({'article_id': articulos_candidatos})
    df_predict['customer_id'] = customer_id
    
    # 4. Enriquecer el DataFrame con las características del usuario y de los productos
    df_predict = pd.merge(df_predict, user_profile, on='customer_id', how='left')
    df_predict = pd.merge(df_predict, final_item_features, on='article_id', how='left')
    
    # 5. Preparar los datos para que coincidan con el formato de entrenamiento
    print("Paso 2: Preparando características para el modelo...")
    # Seleccionar las mismas columnas de características que usamos para entrenar
    X_pred = df_predict.drop(columns=['customer_id', 'article_id'])
    
    # Aplicar la misma codificación ordinal que en el entrenamiento
    # Es crucial usar un encoder 'fittado' o re-codificar de la misma manera
    categorical_features = X_pred.select_dtypes(include=['object', 'category']).columns.tolist()
    # Usamos el mismo OrdinalEncoder que antes, o creamos uno nuevo (para este ejemplo es suficiente)
    from sklearn.preprocessing import OrdinalEncoder
    encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    # Importante: aquí hacemos fit_transform sobre las características de los candidatos
    X_pred[categorical_features] = encoder.fit_transform(X_pred[categorical_features])

    # 6. Alinear las columnas (Paso CRÍTICO para evitar errores)
    # Nos aseguramos de que el DataFrame de predicción tenga exactamente las mismas columnas
    # y en el mismo orden que los datos con los que se entrenó el modelo.
    X_pred_aligned = X_pred.reindex(columns=training_columns, fill_value=0)

    # 7. Realizar la predicción de probabilidades
    print("Paso 3: Calculando puntuaciones de recomendación...")
    scores = modelo.predict_proba(X_pred_aligned)[:, 1]
    
    # 8. Unir puntuaciones y devolver el Top-N
    df_predict['score'] = scores
    recomendaciones = df_predict.sort_values('score', ascending=False).head(n_recommendations)
    
    # Añadir información legible (nombre del producto) para la salida final
    recomendaciones_final = pd.merge(recomendaciones, df_articles[['article_id', 'prod_name']], 
                                     on='article_id', how='left')
    
    print("Paso 4: ¡Recomendaciones generadas!")
    return recomendaciones_final[['article_id', 'prod_name', 'score']]


In [ ]:
# --- 5. EJEMPLO DE USO ---
# Seleccionar un cliente aleatorio de nuestro diccionario de compras para asegurar que tiene historial
lista_de_clientes = list(purchased_items_dict.keys())
cliente_ejemplo = random.choice(lista_de_clientes)

# Generar las recomendaciones para este cliente
recomendaciones_para_cliente = generar_recomendaciones(cliente_ejemplo, modelo)

# Mostrar los resultados de una manera bonita
print(f"\n\n✓✓✓ Top 10 Recomendaciones para el Cliente: {cliente_ejemplo} ✓✓✓\n")
print(recomendaciones_para_cliente.to_string(index=False))